# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml.core
from azureml.core import Workspace, Experiment, Datastore, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.train.sklearn import SKLearn
from azureml.exceptions import ComputeTargetException
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput
# from azureml.train.hyperdrive import *
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

import os
import shutil
import urllib
import pkg_resources
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import logging
import csv
from scipy import stats
from scipy.stats import skew, boxcox_normmax
from scipy.special import boxcox1p
import lightgbm as lgb


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.18.0


## 1. Dataset

### 1.1 Overview

✅ In this notebook we are going to use the Cardiovascular Disease dataset from Kaggle. Cardiovascular Disease dataset is a Kaggle Dataset the containts history of health status of some persons. A group of them suffered a heart attackt. So using this dataset we can train a model in order to predict if a person could suffer a heart attack.

We can download the data from Kaggle page (https://www.kaggle.com/sulianova/cardiovascular-disease-dataset). In this case, I've download the data in the /data directory. So then we have to register this Dataset.

In [2]:
fileCardioData = 'kaggle/cardio_train.csv'
df = pd.read_csv(fileCardioData, encoding='latin')
df.head(2)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1


In [3]:
import os 
dataDir = 'data'

if not os.path.exists(dataDir):
    os.mkdir(dataDir)

fileData = dataDir + "/initialfile.parquet"

df.to_csv(fileData, index=False)

print("Data written to local folder")

Data written to local folder


### 1.2 Upload to Azure Blob

In [4]:
from azureml.core import Workspace

ws = Workspace.from_config()
print("Workspace: " + ws.name, "Region: " + ws.location, sep = '\n')

# Default datastore
default_store = ws.get_default_datastore() 

default_store.upload_files([fileData], 
                           target_path = 'cardio', 
                           overwrite = True, 
                           show_progress = True)

print("Upload completed")

Workspace: quick-starts-ws-126325
Region: southcentralus
Uploading an estimated of 1 files
Uploading data/initialfile.parquet
Uploaded data/initialfile.parquet, 1 files out of an estimated total of 1
Uploaded 1 files
Upload completed


### 1.3 Create and register datasets

In [5]:
from azureml.core import Dataset
cardio_data = Dataset.Tabular.from_delimited_files(default_store.path('cardio/initialfile.parquet'))

In [6]:
cardio_data = cardio_data.register(ws, 'cardio_data')

## 2. Setup Compute

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
amlcompute_cluster_name = "compt-cluster"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
# Define RunConfig for the compute
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

# Create a new runconfig object
aml_run_config = RunConfiguration()

# Use the aml_compute you created above. 
aml_run_config.target = aml_compute

# Enable Docker
aml_run_config.environment.docker.enabled = True

# Use conda_dependencies.yml to create a conda environment in the Docker image for execution
aml_run_config.environment.python.user_managed_dependencies = False

# Specify CondaDependencies obj, add necessary packages
aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['pandas','scikit-learn','numpy'], 
    pip_packages=['azureml-sdk[automl,explain]', 'scipy','lightgbm'])

print ("Run configuration created.")

Run configuration created.


# 3. Prepare Data

### 3.1 Cleaning Data

In [9]:
# initial columns to use
cols_touse = str(['age', 'height', 'weight', 'ap_hi', 'ap_lo',
       'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'cardio']).replace(",", ";")

In [10]:
from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps import PythonScriptStep

# python scripts folder
prepare_data_folder = './scripts'

# Define output after cleansing step
cleaned_data = PipelineData("cleaned_data", datastore=default_store).as_dataset()

print('Cleaning script is in {}.'.format(os.path.realpath(prepare_data_folder)))

# Cleaning step creation
cleaningStep = PythonScriptStep(
    name="Clean Data",
    script_name="clean.py", 
    arguments=["--useful_columns", cols_touse,
               "--output_clean", cleaned_data],
    inputs=[cardio_data.as_named_input('raw_data')],
    outputs=[cleaned_data],
    compute_target=aml_compute,
    runconfig=aml_run_config,
    source_directory=prepare_data_folder,
    allow_reuse=True
)

print("Clean Step created")

Cleaning script is in /mnt/batch/tasks/shared/LS_root/mounts/clusters/compt-inst/code/nd00333_AZMLND_Capstone_Project/scripts.
Clean Step created


### 3.2 Filtering Data

In [11]:
# Define output after merging step
filtered_data = PipelineData("filtered_data", datastore=default_store).as_dataset()

print('Filter script is in {}.'.format(os.path.realpath(prepare_data_folder)))

# filter step creation
# See the filter.py for details about input and output
filterStep = PythonScriptStep(
    name="Filter Data",
    script_name="filter.py", 
    arguments=["--output_filter", filtered_data],
    inputs=[cleaned_data.parse_parquet_files()],
    outputs=[filtered_data],
    compute_target=aml_compute,
    runconfig = aml_run_config,
    source_directory=prepare_data_folder,
    allow_reuse=True
)

print("Filter Step created")

Filter script is in /mnt/batch/tasks/shared/LS_root/mounts/clusters/compt-inst/code/nd00333_AZMLND_Capstone_Project/scripts.
Filter Step created


### 3.3 Transform Data

In [12]:
# Define output after transform step
transformed_data = PipelineData("transformed_data", datastore=default_store).as_dataset()

print('Transform script is in {}.'.format(os.path.realpath(prepare_data_folder)))

# transform step creation
# See the transform.py for details about input and output
transformStep = PythonScriptStep(
    name="Transform Data",
    script_name="transform.py", 
    arguments=["--output_transform", transformed_data],
    inputs=[filtered_data.parse_parquet_files()],
    outputs=[transformed_data],
    compute_target=aml_compute,
    runconfig = aml_run_config,
    source_directory=prepare_data_folder,
    allow_reuse=True
)

print("Transform Step created")

Transform script is in /mnt/batch/tasks/shared/LS_root/mounts/clusters/compt-inst/code/nd00333_AZMLND_Capstone_Project/scripts.
Transform Step created


### 3.4 Split Data into train and test sets

In [13]:
# train and test splits output
output_split_train = PipelineData("output_split_train", datastore=default_store).as_dataset()
output_split_test = PipelineData("output_split_test", datastore=default_store).as_dataset()

print('Data spilt script is in {}.'.format(os.path.realpath(prepare_data_folder)))

# test train split step creation
# See the train_test_split.py for details about input and output
testTrainSplitStep = PythonScriptStep(
    name="Train Test Data Split",
    script_name="train_test_split.py", 
    arguments=["--output_split_train", output_split_train,
               "--output_split_test", output_split_test],
    inputs=[transformed_data.parse_parquet_files()],
    outputs=[output_split_train, output_split_test],
    compute_target=aml_compute,
    runconfig = aml_run_config,
    source_directory=prepare_data_folder,
    allow_reuse=True
)

print("TrainTest Split Step created")

Data spilt script is in /mnt/batch/tasks/shared/LS_root/mounts/clusters/compt-inst/code/nd00333_AZMLND_Capstone_Project/scripts.
TrainTest Split Step created


## 4. HyperDrive 

In [35]:
from azureml.core import Experiment

experiment = Experiment(ws, 'HyperDrive-Pipeline')

print("Experiment created")

Experiment created


### 4.1 HyperDrive config

In [36]:
# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--num_leaves': choice([16, 32, 48, 64]),
        '--max_depth': choice([3, 4, 5, 6]),
        '--min_data_in_leaf': choice([24, 32, 48, 64]),
        '--learning_rate': uniform(0.001, 0.1)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

script_params = {
    '--data_folder': ws.get_default_datastore(),
}

est = Estimator(source_directory=prepare_data_folder,
                #script_params=script_params,
                compute_target=aml_compute,
                entry_script='train_hyperdrive.py',
                pip_packages=['scikit-learn','lightgbm']
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='AUC',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=16,
    max_concurrent_runs=4)

### 4.2 HyperDrive Step

In [37]:
ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics',))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model', model_file = 'outputs/model/model.pkl'))

training_dataset = output_split_train.parse_parquet_files()

hd_step_name='HyperDrive Module'
hd_step = HyperDriveStep(
    name=hd_step_name,
    hyperdrive_config=hyperdrive_config,
    estimator_entry_script_arguments=['--data_folder', ws.get_default_datastore()],
    inputs=[training_dataset],
    outputs=[metrics_data, model_data],
    allow_reuse=True
)

print("HyperDrive step created")

HyperDrive step created


### 4.2 Pipeline

In [38]:
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

pipeline_steps = [hd_step]

pipeline = Pipeline(workspace = ws, steps=pipeline_steps)
print("Pipeline is built.")

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


Pipeline is built.


### 4.3 Experiment

In [39]:
pipeline_run = experiment.submit(pipeline, regenerate_outputs=False)

print("Pipeline submitted for execution.")

Created step HyperDrive Module [51cebcce][d443d577-b41f-443a-88d8-a662313b9134], (This step will run and generate new outputs)
Created step Train Test Data Split [18ec8e09][1a842aac-82e4-4e19-87e7-74e754951604], (This step will run and generate new outputs)
Created step Transform Data [9bf831ed][7faa2068-f739-4fb8-84bf-e12fd7c39120], (This step will run and generate new outputs)
Created step Filter Data [edf64695][35316442-fe02-4f3b-b843-03cf75ec9065], (This step will run and generate new outputs)
Created step Clean Data [8ec78da8][96e02461-2fb3-4115-a2c8-c2025a60bdd2], (This step will run and generate new outputs)
Submitted PipelineRun 10ebedfc-3d87-4bd1-901e-976b6eea8cff
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/HyperDrive-Pipeline/runs/10ebedfc-3d87-4bd1-901e-976b6eea8cff?wsid=/subscriptions/c463503f-66c4-48b5-9bb5-b66fec87c814/resourcegroups/aml-quickstarts-126325/workspaces/quick-starts-ws-126325
Pipeline submitted for execution.


### 4.4 RunDetails

In [40]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [ ]:
# Before we proceed we need to wait for the run to complete.
pipeline_run.wait_for_completion(show_output=False)

PipelineRunId: 10ebedfc-3d87-4bd1-901e-976b6eea8cff
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/HyperDrive-Pipeline/runs/10ebedfc-3d87-4bd1-901e-976b6eea8cff?wsid=/subscriptions/c463503f-66c4-48b5-9bb5-b66fec87c814/resourcegroups/aml-quickstarts-126325/workspaces/quick-starts-ws-126325


### 4.5 Best Model

In [22]:
# View the details of the AutoML run
from azureml.train.hyperdrive.run import HyperDriveRun

hyperdrive_run = HyperDriveRun(experiment=experiment,run_id='HD_b33b9c60-f331-4e4c-bf94-8658bdd30653')
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [23]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print(best_run)
print('Best AUC: {}'.format(best_run_metrics['AUC']))

Run(Experiment: HyperDrive-Pipeline,
Id: HD_b33b9c60-f331-4e4c-bf94-8658bdd30653_0,
Type: azureml.scriptrun,
Status: Completed)
Best AUC: 0.8027723131363702


In [25]:
# functions to download output to local and fetch as dataframe
def get_download_path(download_path, output_name):
    output_folder = os.listdir(download_path + '/azureml')[0]
    path =  download_path + '/azureml/' + output_folder + '/' + output_name
    return path

def fetch_df(step, output_name):
    output_data = step.get_output_data(output_name)    
    download_path = './outputs/' + output_name
    output_data.download(download_path, overwrite=True)
    df_path = get_download_path(download_path, output_name) + '/processed.parquet'
    return pd.read_parquet(df_path)

In [24]:
result_hd_step = pipeline_run.find_step_run(hd_step.name)[0]

In [26]:
output_data = result_hd_step.get_output_data(model_data.name)
output_data

Name,Datastore,Path on Datastore,Produced By PipelineRun,Produced By StepRun
model_data,workspaceblobstore,azureml/bbdfca65-ffdd-4718-bc39-5ece8b053557/model_data,8082c1ca-2ad7-4321-8196-74d11673905d,bbdfca65-ffdd-4718-bc39-5ece8b053557


In [28]:
download_path = './outputs/' + model_data.name
output_data.download(download_path, overwrite=True)

0

In [31]:
parameter_values = best_run.get_details()['runDefinition']['arguments']
parameter_values

['--data_folder',
 '{\n  "name": "workspaceblobstore",\n  "container_name": "azureml-blobstore-81a2e42b-421c-4cf8-86ff-af4675187472",\n  "account_name": "mlstrg126325",\n  "protocol": "https",\n  "endpoint": "core.windows.net"\n}',
 '--learning_rate',
 '0.056924294524143854',
 '--max_depth',
 '5',
 '--min_data_in_leaf',
 '48',
 '--num_leaves',
 '16']

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'your experiment name here'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = <your policy here>

#TODO: Create the different params that you will be using during training
param_sampling = <your params here>

#TODO: Create your estimator and hyperdrive config
estimator = <your estimator here>

hyperdrive_run_config = <your config here?

In [ ]:
#TODO: Submit your experiment

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service